#### Parameter File Overview:
Parameter files can be TSVs or CSVs. The first line of the file should contain column headings. The following headings are required (in any order): mechanism_id, part_id, param_name, param_val (spaces can be substituted for underscores and headings are not case sensitive). mechanism_id is the name of the Mechanism or the kind of mechanism that will use this parameter, for example "transcription" or  "transcription_mm" for Mechalis-Menten transcription would go in this column. part_id refers to the name of the Component that will use this mechanism, for example "ptet" for a tet repressed promoter. param_name refers to the name of the model parameter, for example "ktx", "kb", or "ku". The value of these columns is case sensitive and underscores are different from spaces.

#### Parameter Value Defaulting:
Not all parameters need to have the required headings. The only two required columns are "param_val" and "param_name". BioCRNpyler uses a form of parameter name defaulting discussed below to find default parameters if no exact match is in the config file. This makes it easy to set default parameters for things like "ku" and "ktx" to quickly build models.

#### Parameters inside BioCRNpyler:
Inside of bioCRNpyler, parameters are stored as a dictionary key value pair: (mechanism_name, part_id, param_name) --> param_val. If that particular parameter key cannot be found, the software will default to the following keys: (mechanism_type, part_id, param_name) >> (part_id, param_name) >> (mechanism_name, param_name) >> (mechanism_type, param_name) >>(param_name) and give a warning. As a note, mechanism_name refers to the .name variable of a Mechanism. mechanism_type refers to the .type variable of a Mechanism. Either of these can be used as a mechanism_id. This allows for models to be constructed easily using default parameter values and for parameters to be shared between different Mechanisms and/or Components.

#### Multiple Parameter Files:
Components and Mixtures can both have one more multiple parameter files by passing in a list of filenames instead of a single filename to the parameter_file keyword. Components use parameters loaded from their file(s) before defaulting to the file(s) supplied to a Mixture. The last file in any list will take precedent and overwrite parameter files which were written earlier.

#### Suppressing warnings
To suppress parameter warnings, use the keyword parameter_warnings = False inside a Mixture or Component constructor.

Below is an example csv with all the parameters for a tetR promoter undergoing Michalis Menten transcription and translation.

## 1. The Parameter File
In the following cell we look at an example parameter file that will run with no parameter defaulting.

In [1]:
from biocrnpyler import *
perfect_param_file_name = "Perfect Param File Example.tsv"

#Open and print the parameter file
param_file = open(perfect_param_file_name)
print("****Parameter File****")
print(param_file.read())
param_file.close()


#Create a Regulated Promoter
Ptet = RegulatedPromoter("ptet", regulators=["tetR"], leak=True)
reg_rep_assembly = DNAassembly(name="reporter", promoter=Ptet, rbs="BCD")
tet = Protein("tetR")
components = [reg_rep_assembly, tet]
myMixture = TxTlExtract(name="txtl", parameter_file = perfect_param_file_name, components=components, parameter_warnings=True)


#Print the parameter dictionary created from the file
print("\n****Loaded Parameters****")
for k in myMixture.parameters:
    print("param_dict["+repr(k)+"] = ", myMixture.parameters[k])
    
print("\n****Resulting CRN****")
print(myMixture.compile_crn())


C:\Users\apand\Anaconda3\lib\site-packages\biocrnpyler-0.1-py3.7.egg\biocrnpyler\__init__.py:23: UserWarning: No module named 'fa2'
C:\Users\apand\Anaconda3\lib\site-packages\biocrnpyler-0.1-py3.7.egg\biocrnpyler\__init__.py:24: UserWarning: plotting is disabled because you are missing some libraries


****Parameter File****
mechanism_id	part_id	param_name	param_val	comments
transcription_mm	ptet_tetR	kb	10.	extra columns are okay!
transcription_mm	ptet_tetR	ku	.1	These are the parameters for transcription
transcription_mm	ptet_tetR	ktx	1.		
transcription_mm	ptet	ktx	.1	These are the parameters for transcription leak
transcription_mm	ptet	ku	.1		
transcription_mm	ptet	kb	.1		
one_step_cooperative_binding	ptet_tetR	ku	.1	These are parameters for tetR dimerization and binding with the promoter	
one_step_cooperative_binding	ptet_tetR	kb	.1
one_step_cooperative_binding	ptet_tetR	cooperativity	2
translation_mm	BCD	ktl	2.0	These are parameters for translation
translation_mm	BCD	ku	.25		
translation_mm	BCD	kb	10
rna_degredation_mm		kb	10	These are parameters for RNA degredation. 
rna_degredation_mm		ku	.5	They will be the same for all RNAs because of parameter defaulting.
rna_degredation_mm		kdeg	1.5		

****Loaded Parameters****
param_dict[('transcription_mm', 'ptet_tetR', 'kb')] =  10.0
pa

We will now look at an example of a parameter file that uses defaulting. If you were to fill in this file with full parameter signatures (mechanism_id, part_id, param_name, value), the errors at the bottom of the readout would slowly diminish. However, even without full values the file loads and runs. Although this example uses only the key "param_name" for default values, there exists a heirarchy of keys to allow for shared parameters between different Components and Mechanisms.

The parameter key heirarchy (top takes priority):
1) (mechanism_name, part_id, param_name)
2) (mechanism_type, part_id, param_name)
3) (part_id, param_name)
4) (mechanism_name, param_name)
5) (mechanism_type, param_name)
6) (param_name)

here the column "mechanism_type" can either be a Mechanism's type string Mechanism.type (eg "transcription") or its name string Mechanism.name (eg "transcription_mm"). 

In [2]:
from biocrnpyler import *
default_param_file_name = "Default Param File Example.tsv"

#Open and print the parameter file
param_file = open(default_param_file_name)
print("****Parameter File****")
print(param_file.read())
param_file.close()


#Create a Regulated Promoter
Ptet = RegulatedPromoter("ptet", regulators=["tetR"], leak=True)
reg_rep_assembly = DNAassembly(name="reporter", promoter=Ptet, rbs="BCD")
tet = Protein("tetR")
components = [reg_rep_assembly, tet]

myMixture = TxTlExtract(name="txtl", parameter_file = default_param_file_name, components=components, parameter_warnings=False)

#To Run With Parameter Warnings, change the keyword parameter_warnings = True
#myMixture = TxTlExtract(name="txtl", parameter_file = default_param_file_name, components=components, parameter_warnings=True)


#Print the parameter dictionary created from the file
print("\n****Loaded Parameters****")
for k in myMixture.parameters:
    print("param_dict["+repr(k)+"] = ", myMixture.parameters[k])
    
print("\n****Resulting CRN****")
print(myMixture.compile_crn())

****Parameter File****
param_name	param_val	comments	mechanism_id	part_id
ku	1.0	Any parameter called ku will default to this		
kb	1.0	Default kb		
ktx	2.0	Default ktx		
ktl	3.0	Default ktl		
cooperativity	2.0	Default cooperativity		
kdeg	.5	Default degredation		

****Loaded Parameters****
param_dict[('transcription_mm', 'ptet_tetR', 'kb')] =  10.0
param_dict[('transcription_mm', 'ptet_tetR', 'ku')] =  0.1
param_dict[('transcription_mm', 'ptet_tetR', 'ktx')] =  1.0
param_dict[('transcription_mm', 'ptet', 'ktx')] =  0.1
param_dict[('transcription_mm', 'ptet', 'ku')] =  0.1
param_dict[('transcription_mm', 'ptet', 'kb')] =  0.1
param_dict[('one_step_cooperative_binding', 'ptet_tetR', 'ku')] =  0.1
param_dict[('one_step_cooperative_binding', 'ptet_tetR', 'kb')] =  0.1
param_dict[('one_step_cooperative_binding', 'ptet_tetR', 'cooperativity')] =  2.0
param_dict[('translation_mm', 'BCD', 'ktl')] =  2.0
param_dict[('translation_mm', 'BCD', 'ku')] =  0.25
param_dict[('translation_mm', 'BCD', 'k

### Setting Parameters at the component level
Components can have their own parameter files instead of relying on the parameter files passed into a mixture. Components can also have the mixtures default parameters overwritten with a parameter dictionary. Below, we will create a DNAassembly which has custom parameters loaded in as a dictionary (this works the same as loading them with a file). We will put this in a Mixture with the default parameters from the above example. There are now many fewer parameter warnings as well. This example also helps illustrate how the parameter loading heirarchy works if you examine the final CRNs.

In [3]:
#Create custom parameter dictionary for the ptet promoter. In this case, we will add specific leak parameters and
ra_param_dict = {
    ("transcription_mm", "ku"):33.33, #These parameters will take priority over single key parameters
    ("transcription_mm", "kb"):.3333, 
    ("transcription_mm", "ktx"):3.333, 
    ("transcription_mm", "ptet_leak", "ku"):111.1, #these parameters will take priority over the ones above
    ("transcription_mm", "ptet_leak", "kb"):.1111, 
}

#Use the parameter_file keyword to update the parameters with a file.
#Use the parameters keyword to update the parameters with a dictionary
#If Use both: the dictionary takes precedent to the file if there are conflicts.
Ptet = RegulatedPromoter("ptet", regulators=["tetR"], leak=True, parameters = ra_param_dict)
reg_rep_assembly = DNAassembly(name="reporter", promoter=Ptet, rbs="BCD")
tet = Protein("tetR")
components = [reg_rep_assembly, tet]
myMixture = TxTlExtract(name="txtl", parameter_file = default_param_file_name, components=components, parameter_warnings = False)

myCRN = myMixture.compile_crn()
print(myCRN)

Species = complex_protein_Ribo_rna_reporter, dna_reporter_2x_protein_tetR, dna_reporter, complex_dna_reporter_2x_protein_tetR_protein_RNAP, protein_reporter, protein_RNAP, protein_RNAase, rna_reporter, complex_protein_RNAase_rna_reporter, protein_Ribo, complex_dna_reporter_protein_RNAP, protein_tetR
Reactions = [
	dna_reporter + protein_RNAP <--> complex_dna_reporter_protein_RNAP        massaction: k_f(dna_reporter,protein_RNAP)=0.1*dna_reporter*protein_RNAP k_r(complex_dna_reporter_protein_RNAP)=0.1*complex_dna_reporter_protein_RNAP
	complex_dna_reporter_protein_RNAP --> dna_reporter + rna_reporter + protein_RNAP        massaction: k_f(complex_dna_reporter_protein_RNAP)=0.1*complex_dna_reporter_protein_RNAP
	2.0 protein_tetR + dna_reporter <--> dna_reporter_2x_protein_tetR        massaction: k_f(protein_tetR,dna_reporter)=0.1*protein_tetR^2.0*dna_reporter k_r(dna_reporter_2x_protein_tetR)=0.1*dna_reporter_2x_protein_tetR
	dna_reporter_2x_protein_tetR + protein_RNAP <--> complex_dna_re

### Suppressing warnings:
We can see when default parameters are loaded by toggling the 'parameter_warnings' keyword for a Mixture or a Component. By default this is set to None for Mixtures, which means warnings can be toggled at the Component level. If set to True/False for a Mixture, this will supersede the Component level toggling. The default setting for Component is parameter_warnings = True. Below if you change the parameter warnings for various Components or the Mixture, the warning messages printed will change/disappear.


In [4]:
Ptet = RegulatedPromoter("ptet", regulators=["tetR"], leak=False)
reg_rep_assembly = DNAassembly(name="reporter", promoter=Ptet, rbs="BCD", parameter_warnings = True)
tet = Protein("tetR")
components = [reg_rep_assembly, tet]
myMixture = TxTlExtract(name="txtl", parameter_warnings = None, parameter_file = default_param_file_name, components=components)

myCRN = myMixture.compile_crn()

print(myCRN)

Species = complex_protein_Ribo_rna_reporter, dna_reporter_2x_protein_tetR, dna_reporter, complex_dna_reporter_2x_protein_tetR_protein_RNAP, protein_reporter, protein_RNAP, protein_RNAase, rna_reporter, complex_protein_RNAase_rna_reporter, protein_Ribo, protein_tetR
Reactions = [
	2.0 protein_tetR + dna_reporter <--> dna_reporter_2x_protein_tetR        massaction: k_f(protein_tetR,dna_reporter)=0.1*protein_tetR^2.0*dna_reporter k_r(dna_reporter_2x_protein_tetR)=0.1*dna_reporter_2x_protein_tetR
	dna_reporter_2x_protein_tetR + protein_RNAP <--> complex_dna_reporter_2x_protein_tetR_protein_RNAP        massaction: k_f(dna_reporter_2x_protein_tetR,protein_RNAP)=10.0*dna_reporter_2x_protein_tetR*protein_RNAP k_r(complex_dna_reporter_2x_protein_tetR_protein_RNAP)=0.1*complex_dna_reporter_2x_protein_tetR_protein_RNAP
	complex_dna_reporter_2x_protein_tetR_protein_RNAP --> dna_reporter_2x_protein_tetR + rna_reporter + protein_RNAP        massaction: k_f(complex_dna_reporter_2x_protein_tetR_protei